# Part B: Analayze field of multiple 2 galaxy blends

A catsim-like catalog with multiple two-galaxy blends with different orientations and extent of overlap was created previously. This was then provided as input to the [DESC Weak Lensing Deblending Package](http://weaklensingdeblending.readthedocs.io/en/latest/index.html) (hereafter WLD) and the ouput image analyzed with LSST DM science pipeline (v14.0).  The input and output is analyzed below.

In [ ]:
import numpy as np
from astropy.table import Table, Column
from scipy import spatial
%matplotlib inline
import matplotlib.pyplot as plt
import fitsio


plt.rc('image', origin='lower',
       interpolation='none')
plt.rc('figure.subplot', wspace=0.3)
plt.rc('figure.subplot', hspace=0.3)


### Input Catalog Description:
The catsim-like catalog was created to represent a 576 $\times$ 2880 pixel field, as a combination of 720 postage stamps (each 48 $\times$ 48 pixels). Each postage stamp consits of a two *identical* galaxies with different orientations and with different distance between them. The pair centers have the same y coordinate, thereby difference in centers is just the difference in x cordinate of the centers.
For each galaxy pair, postage stamps are created for:
1. Distance between pair centers are 2, 4 and 6 times the semimajor axis, or overlap extent of 1,2,3 respectively
1. Major axis of the 2 galaxies are oriented at angles of 0, 45 and 90deg, or angle of approach of 0, 45 and 90deg respectively.

For each blend pair from 1) and 2) both galaxies are also rotated by 0,45,90 and 135 degrees. 
Every distinct galaxy pair will have 3 rows of postage stamp; a specific angle of approach, on each row 
Each row of postage stamps has the 4 different rotations of the galaxies for each of the 3 extent of overlaps.
Each distinct galaxy pair has 3 rows and 12 columns of postage stamps.

20 galaxies were selected at random from the One square deg catsim catalog with i band magnitude $<$24 forming the distinct galaxy pairs.

add image of one distinct galaxy pair stamps

In [ ]:
data_dir = "/global/projecta/projectdirs/lsst/groups/WL/projects/wl-btf/two_gal_blend_data/" 

In [ ]:
def get_match(tru_cat, det_cat,
              im_shape, tolerance=5):
    """Returns index of true catalog that are the closest 
    match to the detected catalog centers within the tolerance
    limit.
    """
    z1 = np.zeros((len(tru_cat),2)) 
    z1[:,0] = np.array(tru_cat['dx']/0.2 + (im_shape[1]-1)/2.)
    z1[:,1] = np.array(tru_cat['dy']/0.2 + (im_shape[0]-1)/2.)
    z1_tree = spatial.KDTree(z1)
    z2 = np.zeros((len(det_cat),2)) 
    z2[:,0] = det_cat['base_GaussianCentroid_x']
    z2[:,1] = det_cat['base_GaussianCentroid_y']
    match = z1_tree.query(z2, distance_upper_bound=tolerance)
    return match

def get_grp_size(cat, id_in):
    """Returns number of DM children connected to input source.
    Computes sum of nuber of children of a source's parent and its parent
    and henceforth till an isolated parent is reached"""
    num = 0
    row = cat[cat['id'] == id_in]
    if row['parent'] == 0:
        return 1
    while (row['parent'] != 0):
        new_id = int(row['parent'])
        row = cat[cat['id'] == new_id]
        num += int(row['deblend_nChild'])
    return num

dm_all = Table.read(data_dir + 'mock_gal_pairs_dm.fits',
                    format= 'fits', hdu=1)
purity = 1/(1 + dm_all['base_Blendedness_old'])
col = Column(purity, name="purity")
dm_all.add_column(col)
grp_size = np.zeros(len(dm_all))
for i in range(len(dm_all)):
    grp_size[i] = get_grp_size(dm_all, dm_all["id"][i])
col = Column(grp_size, name="grp_size")
dm_all.add_column(col)



In [ ]:
dm_out = dm_all[dm_all["deblend_nChild"]==0]
dm_image = fitsio.read(data_dir + 'mock_gal_pairs_dm_image.fits',ext=0)
wl_out = Table.read(data_dir + 'mock_gal_pairs_wldeb.fits',
                    format= 'fits', hdu=1)
match = get_match(wl_out, dm_out,dm_image.shape)
select_dm, = np.where((~np.isinf(match[0])) &(~np.isnan(dm_out["base_SdssShape_flux"])))
select_wldeb = match[1][select_dm]
# match_dm = match[1][~np.isinf(match[0])]
print (f"Number of detected children is {len(dm_out)} out of {len(wl_out)} simulated galaxies")
print (f"Number of true matched detections is {len(select_dm)}")




In [ ]:
plt.figure(figsize=[14,10])
plt.subplot(3,3,1)
c=['r', 'g', 'b', 'y', 'k', 'm', 'b', 'y', 'k', 'm']
plt.scatter(wl_out['purity'][select_wldeb],
            dm_out['purity'][select_dm],
            c=dm_out["grp_size"][select_dm],
            vmin=0, vmax=10, alpha=0.3, s=5)
plt.plot([0.5,1], [0.5,1], 'k--')
plt.ylabel("Purity from DM")
plt.xlabel("True purity")
plt.colorbar()
plt.subplot(3,3,2)
plt.scatter(wl_out['flux'][select_wldeb],
            dm_out['base_SdssShape_flux'][select_dm],
           c=dm_out["grp_size"][select_dm],
            vmin=0, vmax=10, alpha=0.3)
plt.colorbar()
plt.ylabel("Flux from DM")
plt.xlabel("True Flux ")
lims = np.array([np.min(dm_out['base_SdssShape_flux'][select_dm]),
        np.max(dm_out['base_SdssShape_flux'][select_dm])])
plt.xlim(lims)
plt.ylim(lims)
plt.plot(lims, lims, 'k--')
plt.plot(lims, 2*lims, 'r--')

plt.subplot(3,3,3)
plt.scatter(wl_out['grp_size'][select_wldeb],
            dm_out['grp_size'][select_dm],
            alpha=0.4)
#plt.plot([0.5,1], [0.5,1], 'k--')
plt.ylabel("grp_size from DM")
plt.xlabel("True grp_size")



In [ ]:
d_flux = ( dm_out['base_SdssShape_flux'][select_dm] - wl_out['flux'][select_wldeb]) / wl_out['flux'][select_wldeb]
plt.scatter(d_flux, wl_out['purity'][select_wldeb]
            c=dm_out["grp_size"][select_dm],
            vmin=0, vmax=10, alpha=0.3)
plt.colorbar()
plt.ylabel("(Flux from DM - True Flux)/ True Flux")
plt.xlabel("True Purity ")

## Exercise:
Q) Plot relation between purity and snr

Q) Plot histograms of shift in centroid estimated by DM?

Q) DM has other blendedness parameters. How do they compare to purity?

Q) Compare true purity of detected objects and undetected objects?